# Deep learning: classifying dog breeds

This project consists of setting up infrastructure to manage the lifecycle of a machine learning model, using AWS resources such as Sagemaker and EC2 to create, train and deploy a pretrained model that can classify images from a dataset containing different dog breeds.

The focus of this proyect is to set up infrastructure that supports the functioning of machine learning models, therefore other tools to monitor and test the model's performance will be used in addition to the activities mentioned above.

In [2]:
#Install needed dependencies
!pip install smdebug


[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: pip install --upgrade pip


In [3]:
#Import libraries
import sagemaker
import boto3
import os
from torchvision.datasets.stanford_cars import StanfordCars
#For Hyperparameter tuning
from sagemaker.tuner import (
    IntegerParameter,
    CategoricalParameter,
    ContinuousParameter,
    HyperparameterTuner,
)
from torchvision import transforms
from sagemaker.pytorch import PyTorch #Estimator

#For debugging and profiling
from sagemaker.debugger import Rule, DebuggerHookConfig, ProfilerRule, rule_configs
from sagemaker.debugger import ProfilerConfig, FrameworkProfile


from smdebug.trials import create_trial
from smdebug.core.modes import ModeKeys
from smdebug.profiler.analysis.notebook_utils.training_job import TrainingJob
from smdebug.profiler.analysis.notebook_utils.timeline_charts import TimelineCharts

import IPython



session = sagemaker.session.Session()
bucket = session.default_bucket()
role = sagemaker.get_execution_role()
region = session.boto_region_name

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2023-02-12 22:36:28.999 pytorch-1-12-cpu-py38-ml-t3-medium-f8be1a063b37f44eb7b009d8cbea:27 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None


/opt/conda/lib/python3.8/site-packages/smdebug-1.0.24b20230106-py3.8.egg/smdebug/profiler/system_metrics_reader.py:78: SyntaxWarning: "is not" with a literal. Did you mean "!="?
/opt/conda/lib/python3.8/site-packages/smdebug-1.0.24b20230106-py3.8.egg/smdebug/profiler/system_metrics_reader.py:78: SyntaxWarning: "is not" with a literal. Did you mean "!="?


## Dataset

This project uses the Stanford Cars dataset, which according to its creators:
> Contains 16,185 images of 196 classes of cars. The data is split into 8,144 training images and 8,041 testing images, where each class has been split roughly in a 50-50 split. Classes are typically at the level of Make, Model, Year, e.g. 2012 Tesla Model S or 2012 BMW M3 coupe.

In [ ]:
python train_model.py --lr .1 --epochs 1 --out-dir ./profile --data-dir s3://sagemaker-us-east-1-793615537614/StanfordCars/

In [4]:
# Command to download and unzip data
local_dir = 'data'
StanfordCars.mirrors = ["https://sagemaker-sample-files.s3.amazonaws.com/datasets/image/StanfordCars/"]
image_dataset = StanfordCars(
    local_dir,
    download=True,
    transform=transforms.Compose(
        [transforms.ToTensor(),
        transforms.Resize([244,244])]
    )
)

In [24]:
#Uploading dataset to S3
prefix = "StanfordCars"

inputs = session.upload_data(path="data", bucket=bucket, key_prefix=prefix)
print("input spec (in this case, just an S3 path): {}".format(inputs))

input spec (in this case, just an S3 path): s3://sagemaker-us-east-1-793615537614/StanfordCars


#### For curious readers:

Datasets included in the pytorch library are listed in this [link](https://pytorch.org/vision/stable/generated/torchvision.datasets.StanfordCars.html#torchvision.datasets.StanfordCars) <br><br>
For the STANFORD CARS dataset, which is used for this project, documentation can be consulted in the [Pytorch website](https://pytorch.org/vision/stable/generated/torchvision.datasets.StanfordCars.html#torchvision.datasets.StanfordCars). Moreover, the original documentation on this dataset can be found [here](https://ai.stanford.edu/~jkrause/cars/car_dataset.html)

## Hyperparameter Tuning
**TODO:** This is the part where you will finetune a pretrained model with hyperparameter tuning. Remember that you have to tune a minimum of two hyperparameters. However you are encouraged to tune more. You are also encouraged to explain why you chose to tune those particular hyperparameters and the ranges.

**Note:** You will need to use the `hpo.py` script to perform hyperparameter tuning.

In [36]:
#Declare hyperparameters that will be tuned
# "lr": ContinuousParameter(0.01, 0.1),
hyperparameter_ranges = {
    "lr": CategoricalParameter([0.01, 0.1]),
    "batch-size": CategoricalParameter([256, 512]),
    'epochs': CategoricalParameter([2,3])
}

In [40]:
estimator = PyTorch(
    entry_point='hpo.py',
    role=role,
    py_version='py38',
    framework_version='1.12',
    instance_count=1,
    instance_type='ml.m5.xlarge'
)

#The training script uses cross entropy loss function, as this is a multiclass classification problem
objective_metric_name = "average test loss"
objective_type = "Minimize"
metric_definitions = [{"Name": "average test loss", "Regex": "Test set: Average loss: ([0-9\\.]+)"}]

tuner = HyperparameterTuner(
    estimator,
    objective_metric_name,
    hyperparameter_ranges,
    metric_definitions,
    max_jobs=8,
    max_parallel_jobs=8,
    objective_type=objective_type
)

In [41]:
#Fit your HP Tuner
tuner.fit({'training': inputs}, wait=True)

No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config
No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config


........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [43]:
best_estimator = tuner.best_estimator()

#Get the hyperparameters of the best trained model
best_estimator.hyperparameters()


2023-02-12 04:37:52 Starting - Preparing the instances for training
2023-02-12 04:37:52 Downloading - Downloading input data
2023-02-12 04:37:52 Training - Training image download completed. Training in progress.
2023-02-12 04:37:52 Uploading - Uploading generated training model
2023-02-12 04:37:52 Completed - Resource retained for reuse


{'_tuning_objective_metric': '"average test loss"',
 'batch-size': '"256"',
 'epochs': '"3"',
 'lr': '"0.01"',
 'sagemaker_container_log_level': '20',
 'sagemaker_estimator_class_name': '"PyTorch"',
 'sagemaker_estimator_module': '"sagemaker.pytorch.estimator"',
 'sagemaker_job_name': '"pytorch-training-2023-02-12-02-31-40-303"',
 'sagemaker_program': '"hpo.py"',
 'sagemaker_region': '"us-east-1"',
 'sagemaker_submit_directory': '"s3://sagemaker-us-east-1-793615537614/pytorch-training-2023-02-12-02-31-40-303/source/sourcedir.tar.gz"'}

## Model Profiling and Debugging
TODO: Using the best hyperparameters, create and finetune a new model

**Note:** You will need to use the `train_model.py` script to perform model profiling and debugging.

In [28]:
# TODO: Set up debugging and profiling rules and hooks
rules = [
    Rule.sagemaker(rule_configs.vanishing_gradient()),
    Rule.sagemaker(rule_configs.overfit()),
    ProfilerRule.sagemaker(rule_configs.ProfilerReport()),
]
hook_config = DebuggerHookConfig(
    hook_parameters={'train.save_interval': "1000", "eval.save_interval": "1000"})
profiler_config = ProfilerConfig(
    system_monitor_interval_millis=5000, framework_profile_params=FrameworkProfile(num_steps=100)
)

In [29]:
#Hyperparameters derived from previous tuning job
best_hyperparameters = {
    'batch-size': '256',
    'epochs': '3',
    'lr': '0.01',
}

estimator = PyTorch(
    entry_point='train_model.py',
    role=role,
    py_version='py38',
    framework_version='1.12',
    instance_count=1,
    instance_type='ml.m5.2xlarge',
    hyperparameters=best_hyperparameters,
    rules=rules,
    debugger_hook_config=hook_config,
    profiler_config=profiler_config,
)

In [31]:
estimator.fit({'training': inputs})

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: pytorch-training-2023-02-13-03-22-46-743


2023-02-13 03:22:47 Starting - Starting the training job...
2023-02-13 03:23:14 Starting - Preparing the instances for trainingVanishingGradient: InProgress
Overfit: InProgress
ProfilerReport: InProgress
......
2023-02-13 03:24:14 Downloading - Downloading input data......
2023-02-13 03:25:15 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2023-02-13 03:25:09,158 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2023-02-13 03:25:09,160 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-02-13 03:25:09,162 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-02-13 03:25:09,172 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2023-02-13 03:25:09,175 sagemaker_pytorch_container.training INFO     Invoking 

In [32]:
from smdebug.profiler.analysis.notebook_utils.training_job import TrainingJob

training_job_name = 'pytorch-training-2023-02-13-03-22-46-743'
tj = TrainingJob(training_job_name, region)
tj.wait_for_sys_profiling_data_to_be_available()

ProfilerConfig:{'S3OutputPath': 's3://sagemaker-us-east-1-793615537614/', 'ProfilingIntervalInMilliseconds': 5000, 'ProfilingParameters': {'DataloaderProfilingConfig': '{"StartStep": 0, "NumSteps": 100, "MetricsRegex": ".*", }', 'DetailedProfilingConfig': '{"StartStep": 0, "NumSteps": 100, }', 'FileOpenFailThreshold': '50', 'HorovodProfilingConfig': '{"StartStep": 0, "NumSteps": 100, }', 'LocalPath': '/opt/ml/output/profiler', 'PythonProfilingConfig': '{"StartStep": 0, "NumSteps": 100, "ProfilerName": "cprofile", "cProfileTimer": "total_time", }', 'RotateFileCloseIntervalInSeconds': '60', 'RotateMaxFileSizeInBytes': '10485760', 'SMDataParallelProfilingConfig': '{"StartStep": 0, "NumSteps": 100, }'}, 'DisableProfiler': False}
s3 path:s3://sagemaker-us-east-1-793615537614/pytorch-training-2023-02-13-03-22-46-743/profiler-output


Profiler data from system is available


In [33]:
from smdebug.profiler.analysis.notebook_utils.timeline_charts import TimelineCharts

system_metrics_reader = tj.get_systems_metrics_reader()
system_metrics_reader.refresh_event_file_list()

view_timeline_charts = TimelineCharts(
    system_metrics_reader,
    framework_metrics_reader=None,
    select_dimensions=["CPU"],
    select_events=["total"],
)

[2023-02-13 04:34:14.177 pytorch-1-12-cpu-py38-ml-t3-medium-f8be1a063b37f44eb7b009d8cbea:27 INFO metrics_reader_base.py:134] Getting 70 event files
select events:['total']
select dimensions:['CPU']
filtered_events:{'total'}
filtered_dimensions:{'CPUUtilization-nodeid:algo-1'}


Profiler analysis code taken from [aws github](https://github.com/aws/amazon-sagemaker-examples/blob/main/sagemaker-debugger/pytorch_profiling/pt-resnet-profiling-single-gpu-single-node.ipynb)

**TODO**: Is there some anomalous behaviour in your debugging output? If so, what is the error and how will you fix it?  
**TODO**: If not, suppose there was an error. What would that error look like and how would you have fixed it?

In [34]:
rule_output_path = estimator.output_path + estimator.latest_training_job.job_name + "/rule-output"
! aws s3 ls {rule_output_path} --recursive
! aws s3 cp {rule_output_path} ./ --recursive
# get the autogenerated folder name of profiler report
profiler_report_name = [
    rule["RuleConfigurationName"]
    for rule in estimator.latest_training_job.rule_job_summary()
    if "Profiler" in rule["RuleConfigurationName"]
][0]
import IPython

IPython.display.HTML(filename=profiler_report_name + "/profiler-output/profiler-report.html")

2023-02-13 04:33:21     363489 pytorch-training-2023-02-13-03-22-46-743/rule-output/ProfilerReport/profiler-output/profiler-report.html
2023-02-13 04:33:20     209519 pytorch-training-2023-02-13-03-22-46-743/rule-output/ProfilerReport/profiler-output/profiler-report.ipynb
2023-02-13 04:33:17        191 pytorch-training-2023-02-13-03-22-46-743/rule-output/ProfilerReport/profiler-output/profiler-reports/BatchSize.json
2023-02-13 04:33:17        199 pytorch-training-2023-02-13-03-22-46-743/rule-output/ProfilerReport/profiler-output/profiler-reports/CPUBottleneck.json
2023-02-13 04:33:17       2025 pytorch-training-2023-02-13-03-22-46-743/rule-output/ProfilerReport/profiler-output/profiler-reports/Dataloader.json
2023-02-13 04:33:17        127 pytorch-training-2023-02-13-03-22-46-743/rule-output/ProfilerReport/profiler-output/profiler-reports/GPUMemoryIncrease.json
2023-02-13 04:33:17        198 pytorch-training-2023-02-13-03-22-46-743/rule-output/ProfilerReport/profiler-output/profiler-re

## Model Deploying

In [42]:
predictor=estimator.deploy(initial_instance_count=1, instance_type='ml.m5.xlarge') # TODO: Add your deployment configuration like instance type and number of instances

INFO:sagemaker:Creating model with name: pytorch-training-2023-02-13-05-39-02-463
INFO:sagemaker:Creating endpoint-config with name pytorch-training-2023-02-13-05-39-02-463
INFO:sagemaker:Creating endpoint with name pytorch-training-2023-02-13-05-39-02-463


-----!

In [43]:
# TODO: Run an prediction on the endpoint
from torch import randint
sample_idx = randint(len(image_dataset), size=(1,)).item()
image, label = image_dataset[sample_idx]



response = predictor.predict(image)
print(response)

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (0) from primary with message "Your invocation timed out while waiting for a response from container primary. Review the latency metrics for each container in Amazon CloudWatch, resolve the issue, and try again.". See https://us-east-1.console.aws.amazon.com/cloudwatch/home?region=us-east-1#logEventViewer:group=/aws/sagemaker/Endpoints/pytorch-training-2023-02-13-05-39-02-463 in account 793615537614 for more information.

In [44]:
# TODO: Remember to shutdown/delete your endpoint once your work is done
predictor.delete_endpoint()

INFO:sagemaker:Deleting endpoint configuration with name: pytorch-training-2023-02-13-05-39-02-463
INFO:sagemaker:Deleting endpoint with name: pytorch-training-2023-02-13-05-39-02-463


In [ ]:
https://pytorch.org/vision/stable/transforms.html

In [ ]:
transformar la imagen en tensor para que sea el formato que pytorch espera.
Y si no funciona, usar las transformaciones en los scripts de entrenamiento para obtener una imagen de 244*244*3
(solamente cambiar el ancho y el largo, ya que todas son a color)

hacer estas mismas transformaciones cuando se mande a llamar el endpoint con la imagen para probar la prediccion